In [234]:
import networkx as nx
from gnpsdata import taskresult
import os
from gnpsdata import workflow_fbmn
import pandas as pd
import csv

In [235]:
task = "cf6e14abf5604f47b28b467a513d3532"

In [236]:
# Downloading raw data from GNPS
def download_graphml(task, output_file):
    taskresult.download_task_resultfile(task, "gnps_molecular_network_graphml/", output_file)

def get_graphml_network(task):
    taskresult.download_task_resultfile(task, "gnps_molecular_network_graphml/", "temp.graphml")

    G = nx.read_graphml("temp.graphml")

    return G

def download_quantification(task, output_file):
    taskresult.download_task_resultfile(task, "quantification_table/", output_file)

def download_metadata(task, output_file):
    taskresult.download_task_resultfile(task, "metadata_merged/", output_file)

def download_mgf(task, output_file):
    taskresult.download_task_resultfile(task, "spectra_reformatted/", output_file)
    
# Qiime2 Data
def download_qiime2(task, output_file):
    taskresult.download_task_resultfile(task, "qiime2_output/qiime2_table.qza", output_file)

def download_qiime2_manifest(task, output_file):
    taskresult.download_task_resultfile(task, "qiime2_output/qiime2_manifest.tsv", output_file)

def download_qiime2_metadata(task, output_file):
    taskresult.download_task_resultfile(task, "qiime2_output/qiime2_metadata.tsv", output_file)

In [237]:
# Download quantification and manifest
os.makedirs("../data", exist_ok=True)
download_quantification(task, "../data/quant.csv")
download_qiime2_manifest(task, "../data/manifest.csv")
# Downloading metadata
workflow_fbmn.download_metadata(task, "../data/unprocessed_metadata.tsv")

# Package installation

In [238]:
!pip install pandas numpy plotly scikit-learn scikit-bio pingouin kaleido ipyfilechooser nbformat

In [239]:
# importing necessary modules
import pandas as pd
import numpy as np
import os
import itertools
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance
from sklearn.decomposition import PCA
import scipy.stats as stats
import pingouin as pg
import skbio # Don't import on Windows!!
from ipyfilechooser import FileChooser
from ipywidgets import interact
import warnings

In [240]:
# Disable warnings for cleaner output, comment out for debugging
warnings.filterwarnings('ignore')

# Blank Removal

In [241]:
# Get folder with data files
result_dir = "../data/"
#Read quant.csv and metadata .tsv
ft = pd.read_csv("../data/quant.csv")
md = pd.read_csv("../data/unprocessed_metadata.tsv", sep = "\t").set_index("filename")

# When cutoff is low, more noise (or background) detected; With higher cutoff, less background detected, thus more features observed
cutoff = 0.1

def inside_levels(df):
    # get all the columns (equals all attributes) -> will be number of rows
    levels = []
    types = []
    count = []
    for col in df.columns:
        types.append(type(df[col][0]))
        levels.append(sorted(set(df[col].dropna())))
        tmp = df[col].value_counts()
        count.append([tmp[levels[-1][i]] for i in range(len(levels[-1]))])
    return pd.DataFrame({"ATTRIBUTES": df.columns, "LEVELS": levels, "COUNT":count, "TYPES": types}, index=range(1, len(levels)+1))
new_md = md.copy() #storing the files under different names to preserve the original files
# remove the (front & tail) spaces, if any present, from the rownames of md
new_md.index = [name.strip() for name in md.index]
# for each col in new_md
# 1) removing the spaces (if any)
# 2) replace the spaces (in the middle) to underscore
# 3) converting them all to UPPERCASE
for col in new_md.columns:
    if new_md[col].dtype == str:
        new_md[col] = [item.strip().replace(" ", "_").upper() for item in new_md[col]]

new_ft = ft.copy() #storing the files under different names to preserve the original files
# changing the index in feature table to contain m/z and RT information
new_ft.index = [f"{id}_{round(mz, 3)}_{round(rt, 3)}" for id, mz, rt in zip(ft["row ID"], ft["row m/z"], ft["row retention time"])]
new_ft.index.name = "CustomIndex"
# drop all columns that are not mzML or mzXML file names
new_ft.drop(columns=[col for col in new_ft.columns if ".mz" not in col], inplace=True)
# remove " Peak area" from column names
new_ft.rename(columns={col: col.replace(" Peak area", "").strip() for col in new_ft.columns}, inplace=True)

if sorted(new_ft.columns) != sorted(new_md.index):
    # print the md rows / ft column which are not in ft columns / md rows and remove them
    ft_cols_not_in_md = [col for col in new_ft.columns if col not in new_md.index]
    new_ft.drop(columns=ft_cols_not_in_md, inplace=True)
    md_rows_not_in_ft = [row for row in new_md.index if row not in new_ft.columns]
    new_md.drop(md_rows_not_in_ft, inplace=True)

new_ft = new_ft.reindex(sorted(new_ft.columns), axis=1) #ordering the ft by its column names
new_md.sort_index(inplace=True) #ordering the md by its row names
list(new_ft.columns) == list(new_md.index)
data = new_md
condition = 2
df = pd.DataFrame({"LEVELS": inside_levels(data).iloc[condition-1]["LEVELS"]})
df.index = [*range(1, len(df)+1)]
#Among the shown levels of an attribute, select the ones to keep
blank_id = 1
#Splitting the data into blanks and samples based on the metadata
md_blank = data[data[inside_levels(data)['ATTRIBUTES'][condition]] == df['LEVELS'][blank_id]]
blank = new_ft[list(md_blank.index)]
md_samples = data[data[inside_levels(data)['ATTRIBUTES'][condition]] != df['LEVELS'][blank_id]]
samples = new_ft[list(md_samples.index)]

blank_removal = samples.copy()

# Getting mean for every feature in blank and Samples
avg_blank = blank.mean(axis=1, skipna=False) # set skipna = False do not exclude NA/null values when computing the result.
avg_samples = samples.mean(axis=1, skipna=False)

# Getting the ratio of blank vs samples
ratio_blank_samples = (avg_blank+1)/(avg_samples+1)

# Create an array with boolean values: True (is a real feature, ratio<cutoff) / False (is a blank, background, noise feature, ratio>cutoff)
is_real_feature = (ratio_blank_samples<cutoff)
blank_removal = samples[is_real_feature.values]
# save to file
entry_id = []
entry_mz = []
entry_time = []
for entryCol in blank_removal.index:
    entry = entryCol.split("_")
    entry_id.append(entry[0])
    entry_mz.append(entry[1])
    entry_time.append(entry[2])
blank_removal.insert(0,"#OTU ID",entry_id,True)
blank_removal.insert(1,"sample_name",entry_mz,True)
blank_removal.insert(2,"abundance",entry_time,True)
blank_removal.reset_index(drop=True, inplace=True)
blank_removal.to_csv(os.path.join(result_dir, "Blanks_Removed.tsv"), sep = "\t", index = False)

In [242]:
# Changing Metadata and Manifest Column name

In [243]:
#read metadata file
metadata = pd.read_csv("../data/unprocessed_metadata.tsv", sep = "\t", index_col=False)
#rename 1st column to "#OTU ID
metadata = metadata.rename(columns={"filename":"#OTU ID"})
#convert back to .tsv
metadata.to_csv('../data/metadata.tsv', sep="\t", index=False)

# Import Into Qiime2
## Convert .tsv to .biom
! source activate qiime2-2019.4 && biom convert -i ../data/gnps_quant/quant.tsv -o quant.biom --to-hdf5

In [244]:
! source activate qiime2-2019.4 && biom convert \
  -i ../data/Blanks_Removed.tsv \
  -o ../data/Blanks_Removed_quant.biom --to-hdf5

In [245]:
! source activate qiime2-2019.4 && qiime tools import \
  --input-path ../data/Blanks_Removed_quant.biom \
  --type 'FeatureTable[Frequency]' \
  --input-format BIOMV210Format \
  --output-path ../data/qiime_table.qza

Imported ../data/Blanks_Removed_quant.biom as BIOMV210Format to ../data/qiime_table.qza


# ANOVA
! source activate qiime2-2019.4 && qiime longitudinal anova \
  --m-metadata-file <path-to-metadata-file.tsv> \
  --p-formula "independent_variable~dependent_variable+dependent_variable" \
  --o-visualization <path-to-visualization.qza>

In [246]:
!source activate qiime2-2019.4 && qiime longitudinal anova \
  --m-metadata-file ../data/metadata.tsv \
  --p-formula "ATTRIBUTE_Year~ATTRIBUTE_Sample_Area+ATTRIBUTE_Latitude" \
  --p-sstype 'I' \
  --o-visualization metadata.qzv

Saved Visualization to: metadata.qzv


# Visualization
Qiime2 visualizations do not work in headless environments, we can view them at https://view.qiime2.org/

In [247]:
# Qiime2 Manipulations Happen after this to mirror the other notebooks

! source activate qiime2-2019.4 && qiime

Usage: qiime [OPTIONS] COMMAND [ARGS]...

  QIIME 2 command-line interface (q2cli)
  --------------------------------------

  To get help with QIIME 2, visit https://qiime2.org.

  To enable tab completion in Bash, run the following command or add it to
  your .bashrc/.bash_profile:

      source tab-qiime

  To enable tab completion in ZSH, run the following commands or add them to
  your .zshrc:

      autoload bashcompinit && bashcompinit && source tab-qiime

Options:
  --version   Show the version and exit.
  --help      Show this message and exit.

Commands:
  info                Display information about current deployment.
  tools               Tools for working with QIIME 2 files.
  dev                 Utilities for developers and advanced users.
  alignment           Plugin for generating and manipulating alignments.
  composition         Plugin for compositional data analysis.
  cutadapt            Plugin for removing adapter sequences, primers, and
                      oth

# Principal Coordinate Analysis (PCoA) & Distance Matrix
! source activate qiime2-2019.4 && first create distance matrix 
 qiime diversity beta \
  --i-table <path-to-feature-table.qza> \
  --p-metric canberra_adkins \
  --output-dir <path-to-distance-matrix.qza> \

In [248]:
! source activate qiime2-2019.4 && qiime diversity beta \
  --i-table ../data/qiime_table.qza \
  --p-metric canberra_adkins \
  --o-distance-matrix ../data/distance_matrix.qza

Saved DistanceMatrix to: ../data/distance_matrix.qza


## PCoA
! source activate qiime2-2019.4 && qiime diversity pcoa \
  --i-distance-matrix <path-to-distance-matrix.qza> \
  --p-number-of-dimensions INTEGER \ #optional
  --o-pcoa <path-to-artifact.qza>

In [249]:
! source activate qiime2-2019.4 && qiime diversity pcoa \
  --i-distance-matrix ../data/distance_matrix.qza \
  --o-pcoa ../data/pcoa.qza

Saved PCoAResults to: ../data/pcoa.qza


# Emperor plot
! source activate qiime2-2019.4 && qiime emperor plot \
--i-pcoa <path_to_pcoa.qza> \
--m-metadata-file <path_to_metadata.tsv \
--o-visualization emperor_qiime2

In [250]:
! source activate qiime2-2019.4 && qiime emperor plot \
  --i-pcoa ../data/pcoa.qza \
  --m-metadata-file ../data/metadata.tsv \
  --o-visualization ../data/emperor_plot \
  --p-ignore-missing-samples

Saved Visualization to: ../data/emperor_plot.qzv
